# LinkedIn Job Postings Analysis

- Not completed yet! Things written here are subject to change.
- If you have problems with `pyspark` auto suggestions, there is a workaround mentioned in this [thread](https://github.com/microsoft/pylance-release/issues/4577).

In [13]:
import os
from pyspark.sql import SparkSession

spark = (
    SparkSession
        .builder
        # local = single thread, local[*] = max threads
        .master('local[*]')
        .appName('Dhika\'s Spark Labs')
        # The default address for Hive metastore
        .config('hive.metastore.uris', 'thrift://localhost:9083')
        # Seems to have no effect when "hive.metastore.uris" is used
        .config('spark.sql.warehouse.dir', 'hdfs://localhost:9000/user/hive/warehouse')
        # Hive must to be enabled to connect from Metabase
        .enableHiveSupport()
        .getOrCreate()
)

# To differentiate between HDFS and real system path
# By default, PySpark will use HDFS path if we enable Hive
# hdfs:// = Hadoop, file:// = real system
def real_path(path):
    return 'file://' + os.path.abspath(path)

In [7]:
# View all databases
# print(spark.sql('SHOW DATABASES').show())
print(spark.catalog.listDatabases())

# View all tables
# print(spark.sql('SHOW TABLES').show())
print(spark.catalog.listTables())

[Database(name='default', catalog='spark_catalog', description='Default Hive database', locationUri='hdfs://localhost:9000/user/hive/warehouse')]
[]


## Read data from source files

In [19]:
import pandas as pd

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [8]:
import os

# Without file extension
files = [
    'data/job_postings',
    'data/company_details/companies',
    'data/company_details/company_industries',
    'data/company_details/company_specialities',
    'data/company_details/employee_counts',
    'data/job_details/benefits',
    'data/job_details/job_industries',
    'data/job_details/job_skills'
]

headers = {}

for file in files:
    if os.path.isfile(f'{file}.csv'):
        # Force all column types as string
        # Auto conversion is inaccurate and will only give me more work
        csv = pd.read_csv(f'{file}.csv', dtype = str)

        # It seems that PySpark can't read complex CSV files correctly
        # Though Pandas seems able to read them just fine
        # As workaround, I use Pandas to export them to JSON first
        csv.to_json(f'{file}.json', orient = 'records')

        # Save the column order for later
        headers[f'{file}.json'] = list(csv.columns)

## Get table overview and join related tables

In [9]:
# Get table columns overview
# Kind of useful to see relationship between tables
for key, value in headers.items():
    print(f'* {key} *')
    print(value)
    print()

* data/job_postings.json *
['job_id', 'company_id', 'title', 'description', 'max_salary', 'med_salary', 'min_salary', 'pay_period', 'formatted_work_type', 'location', 'applies', 'original_listed_time', 'remote_allowed', 'views', 'job_posting_url', 'application_url', 'application_type', 'expiry', 'closed_time', 'formatted_experience_level', 'skills_desc', 'listed_time', 'posting_domain', 'sponsored', 'work_type', 'currency', 'compensation_type']

* data/company_details/companies.json *
['company_id', 'name', 'description', 'company_size', 'state', 'country', 'city', 'zip_code', 'address', 'url']

* data/company_details/company_industries.json *
['company_id', 'industry']

* data/company_details/company_specialities.json *
['company_id', 'speciality']

* data/company_details/employee_counts.json *
['company_id', 'employee_count', 'follower_count', 'time_recorded']

* data/job_details/benefits.json *
['job_id', 'inferred', 'type']

* data/job_details/job_industries.json *
['job_id', 'indu

In [14]:
job_postings = 'data/job_postings.json'
df_jp = spark.read.json(real_path(job_postings))

# Reading CSV directly using PySpark may cause issues (see above)
# Single multiline row can be mistreated as multiple rows
""" # header = get column name from CSV header
# inferSchema = auto detect column data type when possible
df_jp = (
    spark.read.options(
        header = True,
        inferSchema = True
    ).csv('data/job_postings.csv')
) """

# Restore original column order (the default is ascending)
df_jp = df_jp.select(*headers[job_postings])

# Print column names and types
# No need since all types are string
# df_jp.printSchema()

# Compare all rows vs distinct rows
print(df_jp.count())
print(df_jp.distinct().count())

15886


15886


We can either use `print(xxx.show())` or `display(xxx.toPandas())` to print the output

In [21]:
# Treat current CSV as temporary SQL table
# You can then execute SQL commands on this table
df_jp.createOrReplaceTempView("job_postings")

# We can also use "select distinct" to return only unique rows,
# but all rows are already unique based on previous result
display(spark.sql('SELECT * FROM job_postings').toPandas())
display(spark.sql('SELECT COUNT(*) FROM job_postings').toPandas())

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,applies,original_listed_time,remote_allowed,views,job_posting_url,application_url,application_type,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,85008768,None,Licensed Insurance Agent,While many industries were hurt by the last fe...,52000,None,45760,YEARLY,Full-time,"Chico, CA",None,1.69E+12,None,5,https://www.linkedin.com/jobs/view/85008768/?t...,None,ComplexOnsiteApply,1.71E+12,None,None,None,1.69E+12,None,1,FULL_TIME,USD,BASE_SALARY
1,133114754,77766802,Sales Manager,Are you a dynamic and creative marketing profe...,None,None,None,None,Full-time,"Santa Clarita, CA",None,1.69E+12,None,None,https://www.linkedin.com/jobs/view/133114754/?...,None,ComplexOnsiteApply,1.70E+12,None,None,None,1.69E+12,None,0,FULL_TIME,None,None
2,133196985,1089558,Model Risk Auditor,Join Us as a Model Risk Auditor – Showcase You...,None,None,None,None,Contract,"New York, NY",1,1.69E+12,None,17,https://www.linkedin.com/jobs/view/133196985/?...,None,ComplexOnsiteApply,1.70E+12,None,None,None,1.69E+12,None,0,CONTRACT,None,None
3,381055942,96654609,Business Manager,Business ManagerFirst Baptist Church ForneyFor...,None,None,None,None,Full-time,"Forney, TX",None,1.69E+12,None,None,https://www.linkedin.com/jobs/view/381055942/?...,None,ComplexOnsiteApply,1.70E+12,None,None,None,1.69E+12,None,0,FULL_TIME,None,None
4,529257371,1244539,NY Studio Assistant,YOU COULD BE ONE OF THE MAGIC MAKERS\nKen Fulk...,None,None,None,None,Full-time,"New York, NY",None,1.69E+12,None,2,https://www.linkedin.com/jobs/view/529257371/?...,None,ComplexOnsiteApply,1.71E+12,None,None,None,1.69E+12,None,1,FULL_TIME,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15881,3701373516,74718032,Sanitation Technician,"Location:\n\nWest Columbia, SC, US, 29172\n\n2...",None,None,None,None,Part-time,"West Columbia, SC",None,1.69E+12,None,1,https://www.linkedin.com/jobs/view/3701373516/...,https://aspirebakeriescareers.com/job/West-Col...,OffsiteApply,1.70E+12,None,Entry level,None,1.69E+12,aspirebakeriescareers.com,0,PART_TIME,None,None
15882,3701373522,38897,Unit Secretary,Job Title: Unit Secretary\nDepartment: Nursing...,None,None,None,None,Full-time,"Teaneck, NJ",2,1.69E+12,None,7,https://www.linkedin.com/jobs/view/3701373522/...,https://recruiting.ultipro.com/HOL1005HNMC/Job...,OffsiteApply,1.70E+12,None,Entry level,None,1.69E+12,recruiting.ultipro.com,0,FULL_TIME,None,None
15883,3701373523,38897,"Radiology Aide, Perdiem","Job Title: Radiology Aide, Perdiem\nDepartment...",None,None,None,None,Part-time,"Teaneck, NJ",None,1.69E+12,None,3,https://www.linkedin.com/jobs/view/3701373523/...,https://recruiting.ultipro.com/HOL1005HNMC/Job...,OffsiteApply,1.70E+12,None,Entry level,None,1.69E+12,recruiting.ultipro.com,0,PART_TIME,None,None
15884,3701373524,2623,MRI Manager,Grade 105\nJob Type: Officer of Administration...,135000,None,110000,YEARLY,Full-time,"New York, NY",None,1.69E+12,None,10,https://www.linkedin.com/jobs/view/3701373524/...,https://opportunities.columbia.edu/jobs/mri-ma...,OffsiteApply,1.70E+12,None,Mid-Senior level,None,1.69E+12,opportunities.columbia.edu,0,FULL_TIME,USD,BASE_SALARY


,count(1)
0,15886


In [ ]:
# # Unimportant columns
# filter_cols = [
#     'description',
#     'original_listed_time',
#     'application_url',
#     'job_posting_url',
#     'expiry',
#     'closed_time',
#     'listed_time',
#     'posting_domain',
#     'sponsored',
#     'work_type'
# ]

# # FIXME The "except" SQL query doesn't work
# # Using pythonic way "not in" as workaround
# filter_cols = [ i for i in df_jp.schema.names if i not in filter_cols ]
# filter_cols = ','.join(filter_cols)

# spark.sql(f'SELECT {filter_cols} FROM job_postings').toPandas()

In [ ]:
spark.sql(
"""CREATE TABLE jobs (
    id Int,
    company String,
    industry String,
    title String,
    experience String,
    salary Int,
    currency String,
    work_type String,
    location String,
    benefit String
)""")

In [ ]:
spark.sql('SELECT * FROM job_postings WHERE LOWER(title) LIKE "%data%"').toPandas()

,job_id,company_id,title,description,max_salary,med_salary,min_salary,pay_period,formatted_work_type,location,...,expiry,closed_time,formatted_experience_level,skills_desc,listed_time,posting_domain,sponsored,work_type,currency,compensation_type
0,3586162459,69642092,Teradata Developer,Duration: 6-12+ Months\nOverview of Role:Indiv...,None,None,None,None,Contract,United States,...,1.70E+12,None,None,None,1.69E+12,None,0,CONTRACT,None,None
1,3690692186,61242,Seasonal Payroll/Data Entry Clerk,Universal Screen Arts' specialty is marketing ...,None,None,None,None,Temporary,"Hudson, OH",...,1.71E+12,None,None,None,1.69E+12,None,0,TEMPORARY,None,None
2,3691795980,7573454,Data Engineer,"Job Description:\n• Design, develop, and launc...",None,None,None,None,Contract,United States,...,1.70E+12,None,None,None,1.69E+12,None,0,CONTRACT,None,None
3,3692302089,37768,Data Scientist/ Product Analyst,Looking for candidates with 4+ years’ experien...,80,None,70,HOURLY,Contract,"San Francisco, CA",...,1.70E+12,None,Mid-Senior level,None,1.69E+12,None,0,CONTRACT,USD,BASE_SALARY
4,3692363778,2474970,Data Analytics Consultant,About the CompanyDiLytics is a leading Informa...,None,None,None,None,Full-time,"Sacramento, CA",...,1.70E+12,None,None,None,1.69E+12,None,0,FULL_TIME,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376,3701369746,39203,Data Scientist / Operations Research Analyst,LinQuest is seeking a Data Scientist / Operati...,160000,None,100000,YEARLY,Full-time,"Colorado Springs, CO",...,1.70E+12,None,Entry level,None,1.69E+12,recruiting2.ultipro.com,0,FULL_TIME,USD,BASE_SALARY
377,3701371901,2848937,Data Engineering Product Lead (Hybrid),Job Description\nThe Data Engineering Product ...,304500,None,193440,YEARLY,Full-time,"West Point, PA",...,1.70E+12,None,None,None,1.69E+12,jobs.msd.com,1,FULL_TIME,USD,BASE_SALARY
378,3701372446,2113831,OCM Data Analyst (Remote),GovCIO is looking for an experienced Data Anal...,88000,None,85000,YEARLY,Full-time,"Fairfax, VA",...,1.70E+12,None,Entry level,None,1.69E+12,careers-govcio.icims.com,0,FULL_TIME,USD,BASE_SALARY
379,3701372789,163578,Senior Supply Chain Data Analyst,Job Description:\nOur Sr. Supply Chain Data An...,108000,None,76000,YEARLY,Full-time,"Irvine, CA",...,1.70E+12,None,None,None,1.69E+12,edwards.wd5.myworkdayjobs.com,1,FULL_TIME,USD,BASE_SALARY


In [ ]:
# tables = [ i.name for i in spark.catalog.listTables() ]
# print(tables)

# for table in tables:
#     spark.catalog.dropTempView(table)